In [5]:
import sys
import importlib.util
print("PY:", sys.executable)  # this is the interpreter your notebook is using

# install into THIS interpreter:
import subprocess, shlex
subprocess.check_call(shlex.split(f'{sys.executable} -m pip install -U "pinecone==6.0.1" "langchain-pinecone==0.2.12"'))

# verify modules are visible to THIS interpreter
print("pinecone spec:", importlib.util.find_spec("pinecone"))
print("pinecone_plugins spec:", importlib.util.find_spec("pinecone_plugins"))  # should be None


PY: /opt/miniconda3/envs/mchatbot/bin/python
  Using cached pinecone-6.0.1-py3-none-any.whl.metadata (8.8 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
Using cached pinecone-6.0.1-py3-none-any.whl (421 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pinecone]
pinecone spec: ModuleSpec(name='pinecone', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1076de800>, origin='/opt/miniconda3/envs/mchatbot/lib/python3.10/site-packages/pinecone/__init__.py', submodule_search_locations=['/opt/miniconda3/envs/mchatbot/lib/python3.10/site-packages/pinecone'])
pinecone_plugins spec: None


In [6]:
from pinecone import Pinecone, ServerlessSpec
import pinecone
print("✅ pinecone version:", pinecone.__version__)

pc = Pinecone(api_key="YOUR_API_KEY")
print("✅ client OK")


✅ pinecone version: 6.0.1
✅ client OK


/opt/miniconda3/envs/mchatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone 
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

/opt/miniconda3/envs/mchatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
print(sys.executable)


/opt/miniconda3/envs/mchatbot/bin/python


In [3]:
PINECONE_API_KEY = "pcsk_4fq8QP_J7DL8gSFDwNiVrt6gzhHdFdaaTvpNNsmWxsgCijdh7QojxQVCzVYtSBoqABKNdZ"
PINECONE_API_ENV = "us-east-1"

In [4]:
from langchain.document_loaders import DirectoryLoader, PyMuPDFLoader

def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyMuPDFLoader   # lowercase 'loader_cls'
    )
    documents = loader.load()
    return documents


In [5]:
extracted_data = load_pdf("data/")

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunks : ", len(text_chunks))

length of my chunks :  5777


In [8]:
#Download embedding model
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
import sys, subprocess, shlex
py = sys.executable

# install modern, mutually-compatible versions
subprocess.check_call(shlex.split(f'{py} -m pip install -U "transformers>=4.45.0" "sentence-transformers>=3.0.0" "tokenizers>=0.19.1" "huggingface-hub>=0.33.4" "langchain-huggingface>=0.3.1"'))


0

In [10]:
# embeddings = download_hugging_face_embedding()

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
print(len(embeddings.embed_query("test")))  # 384


384


In [11]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
#testing - Converting hello world into vectos and the dimentionality of that  vectore is 384
result_query = embeddings.embed_query("Hello world")
print("Result  query", len(result_query))

Result  query 384


In [13]:
PINECONE_API_KEY = "pcsk_4fq8QP_J7DL8gSFDwNiVrt6gzhHdFdaaTvpNNsmWxsgCijdh7QojxQVCzVYtSBoqABKNdZ"
PINECONE_API_ENV = "us-east-1"

In [14]:
# #Initializing the Pinecone
# pinecone.init(api_key = PINECONE_API_KEY,
#                environment = PINECONE_API_ENV)

# index_name = "medical-chatbot"

# #Creating Embeddings for Each of The Text Chunks & storing
# docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [15]:
from pinecone import Pinecone, ServerlessSpec

INDEX_NAME = "medical-chatbot"
REGION = "us-east-1"                  # from your dashboard
DIM = 384                             # all-MiniLM-L6-v2

pc = Pinecone(api_key=PINECONE_API_KEY)

# create if missing
if INDEX_NAME not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=INDEX_NAME,
        dimension=DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region=REGION),
    )

index = pc.Index(INDEX_NAME)
print("Indexes:", [i["name"] for i in pc.list_indexes()])


Indexes: ['medical-chatbot']


In [16]:
from pinecone import Pinecone
import numpy as np

print("pinecone version:", __import__("pinecone").__version__)
pc = Pinecone(api_key=PINECONE_API_KEY)
print("indexes:", [i["name"] for i in pc.list_indexes()])

index = pc.Index("medical-chatbot")
print("stats BEFORE:", index.describe_index_stats())


pinecone version: 6.0.1
indexes: ['medical-chatbot']
stats BEFORE: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5779}},
 'total_vector_count': 5779,
 'vector_type': 'dense'}


In [17]:
# --- Step 1: Prepare text chunks ---
texts = [t.page_content for t in text_chunks]
ids = [f"doc-{i}" for i in range(len(texts))]

# --- Step 2: Generate embeddings ---
vectors = embeddings.embed_documents(texts)
print("✅ Embedding dimensions:", {len(v) for v in vectors})  # should show {384}

# --- Step 3: Upsert into Pinecone ---
from tqdm import tqdm

BATCH_SIZE = 100
for i in tqdm(range(0, len(vectors), BATCH_SIZE)):
    batch = [
        {
            "id": ids[j],
            "values": [float(x) for x in vectors[j]],  # ensure pure floats
            "metadata": {"text": texts[j]},
        }
        for j in range(i, min(i + BATCH_SIZE, len(vectors)))
    ]
    index.upsert(vectors=batch)

# --- Step 4: Verify ---
print("Index stats:", index.describe_index_stats())


✅ Embedding dimensions: {384}


100%|██████████| 58/58 [00:52<00:00,  1.11it/s]

Index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5779}},
 'total_vector_count': 5779,
 'vector_type': 'dense'}


In [18]:
import os

# Make sure these are set

os.environ["PINECONE_API_KEY"] = "pcsk_4fq8QP_J7DL8gSFDwNiVrt6gzhHdFdaaTvpNNsmWxsgCijdh7QojxQVCzVYtSBoqABKNdZ"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"

from langchain_pinecone import PineconeVectorStore

index_name = "medical-chatbot"

# connect to your existing Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

query = "What are Allergies?"

docs = docsearch.similarity_search(query, k=3)

for i, d in enumerate(docs):
    print(f"\n🔹Result {i+1}:\n{d.page_content[:300]} ...")


/opt/miniconda3/envs/mchatbot/lib/python3.10/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore



🔹Result 1:
reaction. Allergic rhinitis is characterized by an itchy,
runny nose, often with a scratchy or irritated throat due
to post-nasal drip. Inflammation of the thin membrane
covering the eye (allergic conjunctivitis) causes redness,
irritation, and increased tearing in the eyes. Asthma caus-
es wheezing ...

🔹Result 2:
reactions is triggered by harmless, everyday substances.
This is the condition known as allergy, and the offend-
ing substance is called an allergen. Common inhaled
allergens include pollen, dust, and insect parts from tiny
house mites. Common food allergens include nuts, fish,
and milk.
Allergic re ...

🔹Result 3:
to commonly encountered environmental substances.
Purpose
Allergy is a reaction of the immune system. Nor-
mally, the immune system responds to foreign microor-
ganisms and particles, like pollen or dust, by producing
specific proteins called antibodies that are capable of
binding to identifying mol ...


In [19]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [20]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [21]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [22]:
retriever = docsearch.as_retriever(search_kwargs={"k": 2})

In [23]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [24]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

/var/folders/v9/t1m3g_0n54b7qq3hf262vsch0000gn/T/ipykernel_87869/4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  There are no known allergens in the brain or nervous system. Therefore, it is unlikely that a person's allergy would cause an autopsy to be performed differently than normal. The protocol for performing an autopsy is standardized and follows strict guidelines to ensure accuracy and completeness of the examination.


In [1]:
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.llms import CTransformers
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from pinecone import Pinecone, ServerlessSpec
from tqdm import tqdm
import os

# Configuration
PINECONE_API_KEY = "pcsk_4fq8QP_J7DL8gSFDwNiVrt6gzhHdFdaaTvpNNsmWxsgCijdh7QojxQVCzVYtSBoqABKNdZ"
INDEX_NAME = "medical-chatbot"
REGION = "us-east-1"
DIM = 384

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["PINECONE_ENVIRONMENT"] = REGION

# 1. Load PDF documents
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyMuPDFLoader
    )
    documents = loader.load()
    return documents

# 2. Split text into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, 
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

# 3. Load documents and create chunks
extracted_data = load_pdf("data/")
text_chunks = text_split(extracted_data)
print(f"Length of my chunks: {len(text_chunks)}")

# 4. Initialize embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
print(f"Embedding dimension: {len(embeddings.embed_query('test'))}")

# 5. Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create index if it doesn't exist
if INDEX_NAME not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=INDEX_NAME,
        dimension=DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region=REGION),
    )

index = pc.Index(INDEX_NAME)
print(f"Current index stats: {index.describe_index_stats()}")

# 6. Prepare and upsert vectors to Pinecone
texts = [t.page_content for t in text_chunks]
ids = [f"doc-{i}" for i in range(len(texts))]

# Generate embeddings
vectors = embeddings.embed_documents(texts)
print(f"✅ Embedding dimensions: {len(vectors[0])}")

# Upsert in batches
BATCH_SIZE = 100
for i in tqdm(range(0, len(vectors), BATCH_SIZE), desc="Upserting to Pinecone"):
    batch = [
        {
            "id": ids[j],
            "values": [float(x) for x in vectors[j]],
            "metadata": {"text": texts[j]},
        }
        for j in range(i, min(i + BATCH_SIZE, len(vectors)))
    ]
    index.upsert(vectors=batch)

print(f"Final index stats: {index.describe_index_stats()}")

# 7. Create vector store from existing index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME,
    embedding=embeddings
)

# 8. Test similarity search
query = "What are Allergies?"
docs = docsearch.similarity_search(query, k=3)

print("\n🔍 Test Query Results:")
for i, d in enumerate(docs):
    print(f"\n🔹 Result {i+1}:\n{d.page_content[:300]}...")

# 9. Setup the QA chain
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

# 10. Initialize LLM
llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        'max_new_tokens': 512,
        'temperature': 0.8
    }
)

# 11. Create RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

# 12. Interactive loop
print("\n💬 Medical Chatbot Ready! (Type 'quit' to exit)")
while True:
    user_input = input("\nInput Prompt: ")
    
    if user_input.lower() in ['quit', 'exit', 'q']:
        print("Goodbye!")
        break
    
    if not user_input.strip():
        continue
    
    try:
        result = qa.invoke({"query": user_input})
        print(f"\n✅ Response: {result['result']}")
    except Exception as e:
        print(f"❌ Error: {str(e)}")

/opt/miniconda3/envs/mchatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/miniconda3/envs/mchatbot/lib/python3.10/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


Length of my chunks: 5777
Embedding dimension: 384
Current index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5779}},
 'total_vector_count': 5779,
 'vector_type': 'dense'}
✅ Embedding dimensions: 384


Upserting to Pinecone: 100%|██████████| 58/58 [00:51<00:00,  1.13it/s]


Final index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 5779}},
 'total_vector_count': 5779,
 'vector_type': 'dense'}

🔍 Test Query Results:

🔹 Result 1:
reaction. Allergic rhinitis is characterized by an itchy,
runny nose, often with a scratchy or irritated throat due
to post-nasal drip. Inflammation of the thin membrane
covering the eye (allergic conjunctivitis) causes redness,
irritation, and increased tearing in the eyes. Asthma caus-
es wheezing...

🔹 Result 2:
reactions is triggered by harmless, everyday substances.
This is the condition known as allergy, and the offend-
ing substance is called an allergen. Common inhaled
allergens include pollen, dust, and insect parts from tiny
house mites. Common food allergens include nuts, fish,
and milk.
Allergic re...

🔹 Result 3:
to commonly encountered environmental substances.
Purpose
Allergy is a reaction of the immune system. Nor-
mally, the immune system responds to fore